# Talent list processing
Grab the IdleonEfficiency talent list and merge it with our own

## Imports

### IdleonEfficiency
Import `TalentTreeRepo.ts` from the IdleonEfficiency repo and place it next to the notebook file :  https://github.com/Sludging/idleon-efficiency/blob/main/data/domain/data/TalentTreeRepo.ts

In [6]:
from pathlib import Path
tsData = Path('TalentTreeRepo.ts').read_text()

We then do a little cleanup to convert the TS file into a python dict

This might be higly dependent on the file version, so tweak it as you need

In [7]:
import json

tsData = tsData.replace("import { ActiveTalentModel } from \'../model/activeTalentModel\';\nimport { TalentModel } from \'../model/talentModel\';\nimport { TalentTreeModel } from \'../model/talentTreeModel\';\n\nexport class TalentTreeBase { constructor(public id: string, public data: TalentTreeModel) { } }\n\n\n\nexport const initTalentTreeRepo = () => {\n    return [", "{")
tsData = tsData.replace("new TalentTreeBase(", "")
tsData = tsData.replace(", <TalentTreeModel>", ": ")
tsData = tsData.replace("<TalentModel>", "")
tsData = tsData.replace("<ActiveTalentModel>", "")
tsData = tsData.replace("undefined", "null")
tsData = tsData.replace(")", "")
tsData = tsData.replace("]", "}")
tsData = tsData[:-2] # Strips the extra chars at the end

# print(txt[255100:])
IdleonEfficiencyDict = json.loads(tsData)


### IdleonAutoReviewBot
We now merge our data

In [8]:
import sys
sys.path.insert(1, '/workspaces/IdleOnAutoReviewBot/mysite')
from consts.consts_w3 import combat_talentsDict, skill_talentsDict

all_talentsDict = {}

for categoryName in IdleonEfficiencyDict:
    talents = IdleonEfficiencyDict[categoryName]["talents"]
    for talentName in talents:
        talent = talents[talentName]
        all_talentsDict[talent["skillIndex"]] = talent
        match categoryName:
            case "Beginner" | "Rage Basics" | "Calm Basics"  |"Savvy Basics":
                all_talentsDict[talent["skillIndex"]]["subClass"] = "Tab 1"
            case "Journeyman" | "Warrior" | "Mage" | "Archer":
                all_talentsDict[talent["skillIndex"]]["subClass"] = "Tab 2"
            case "Maestro" | "Barbarian" | "Squire" | "Shaman" | "Wizard" | "Bowman"| "Hunter":
                all_talentsDict[talent["skillIndex"]]["subClass"] = "Tab 3"
            case "Voidwalker" | "Blood Berserker" | "Divine Knight" | "Bubonic Conjuror" | "Elemental Sorcerer" | "Siege Breaker"| "Beast Master":
                all_talentsDict[talent["skillIndex"]]["subClass"] = "Tab 4"
            case "Death Bringer" | "Wind Walker" | "Arcane Cultist":
                all_talentsDict[talent["skillIndex"]]["subClass"] = "Tab 5"
            case "Special Talent 1" | "Special Talent 2" | "Special Talent 3" | "Special Talent 4" | "Special Talent 5":
                all_talentsDict[talent["skillIndex"]]["subClass"] = categoryName
            case _:
                raise Exception(categoryName)  
                
        
autoReviewBotTalentsDict = combat_talentsDict | skill_talentsDict
for skillCategoryName in autoReviewBotTalentsDict:
    skillCategory = autoReviewBotTalentsDict[skillCategoryName]
    for priority in skillCategory:
        priorityGroup = skillCategory[priority]
        for  skillId in priorityGroup:
            skill = priorityGroup[skillId]
            #print(skillCategoryName, priority, skillId, skill["Name"], all_talentsDict[skillId]["name"])
            all_talentsDict[skillId]["category"] = skillCategoryName
            all_talentsDict[skillId]["libraryPriority"] = priority
            if "Hardcap" in skill:
                all_talentsDict[skillId]["hardcap"] = skill["Hardcap"]
            if "Optimal" in skill:
                all_talentsDict[skillId]["optimal"] = skill["Optimal"]

print(all_talentsDict)

{0: {'name': 'Health Booster', 'description': 'Increases Max HP by {', 'x1': 1, 'x2': 0.15, 'funcX': 'add', 'y1': 0, 'y2': 0, 'funcY': 'txt', 'lvlUpText': '+{ Hp', 'skillIndex': 0, 'activeData': None, 'subClass': 'Tab 1', 'category': 'Mining', 'libraryPriority': 'Low'}, 1: {'name': 'Mana Booster', 'description': 'Increases Max MP by {, which is used to cast Class Attacks', 'x1': 1, 'x2': 0.1, 'funcX': 'add', 'y1': 0, 'y2': 0, 'funcY': 'txt', 'lvlUpText': '+{ Mp', 'skillIndex': 1, 'activeData': None, 'subClass': 'Tab 1', 'category': 'Choppin', 'libraryPriority': 'Low'}, 8: {'name': 'Star Player', 'description': 'Gives { Star Talent Points. Star Talents are found later in the game', 'x1': 1, 'x2': 0, 'funcX': 'add', 'y1': 0, 'y2': 0, 'funcY': 'txt', 'lvlUpText': '+1 Star Talent Point', 'skillIndex': 8, 'activeData': None, 'subClass': 'Tab 1', 'category': 'Utility', 'libraryPriority': 'Lowest'}, 9: {'name': 'Bucklered Up', 'description': 'Increases Total Defense by +{%', 'x1': 40, 'x2': 6

Export the results to a file

In [9]:
from pprint import pprint
with open('all_talentsDict.py', 'w') as f:
    f.write('# auto-generated by talents_processing.ipynb. Last done in v2.41\n')
    f.write('all_talentsDict = ')
    pprint(all_talentsDict, stream=f)

Check if the generated file is valid


In [10]:
from all_talentsDict import all_talentsDict

print(all_talentsDict)

{0: {'activeData': None, 'category': 'Mining', 'description': 'Increases Max HP by {', 'funcX': 'add', 'funcY': 'txt', 'libraryPriority': 'Low', 'lvlUpText': '+{ Hp', 'name': 'Health Booster', 'skillIndex': 0, 'subClass': 'Tab 1', 'x1': 1, 'x2': 0.15, 'y1': 0, 'y2': 0}, 1: {'activeData': None, 'category': 'Choppin', 'description': 'Increases Max MP by {, which is used to cast Class Attacks', 'funcX': 'add', 'funcY': 'txt', 'libraryPriority': 'Low', 'lvlUpText': '+{ Mp', 'name': 'Mana Booster', 'skillIndex': 1, 'subClass': 'Tab 1', 'x1': 1, 'x2': 0.1, 'y1': 0, 'y2': 0}, 5: {'activeData': None, 'category': 'Maestro', 'description': 'Increases Base Weapon Power by {. This increases damage!!', 'funcX': 'add', 'funcY': '_', 'libraryPriority': 'Low', 'lvlUpText': '+{ Base Weapon Power', 'name': 'Sharpened Axe', 'skillIndex': 5, 'subClass': 'Tab 1', 'x1': 0.25, 'x2': 0, 'y1': 0, 'y2': 0}, 6: {'activeData': None, 'category': 'Maestro', 'description': 'Boosts Damage dealt to all monsters by {%'